### Homework 2 - Shany Herskovits (040379604) and Ofir Nahshon (038683363)


# Producers Notebook

## First run the consumers notebook and then this notebook



In [1]:
from pyspark import SparkContext
from confluent_kafka import Producer
import random
import time


# Load the file into an RDD and create key-value pairs (word, sentiment)
file_path = "AFINN-111.txt"
sentiments_rdd = sc.textFile(file_path).map(lambda line: tuple(line.split('\t')))

# Initialize Kafka Producer
producer = Producer({'bootstrap.servers': 'localhost:9092'}) 

# Set to keep track of already sent words
sent_words = set()

def send_sentiments():
    global sentiments_rdd, sent_words
    
    # Filter out already sent words
    unsent_rdd = sentiments_rdd.filter(lambda kv: kv[0] not in sent_words)
    
    # Collect data and pick 100 random keys and values
    if unsent_rdd.count() == 0:
        print("All words have been sent. Exiting.")
        return False  # Exit condition if all words have been sent
    
    # Randomly select 100 words
    batch = unsent_rdd.takeSample(withReplacement=False, num=100, seed=1)
    
    # Send each key-value pair to Kafka and mark it as sent
    for word, sentiment in batch:
        producer.produce('sentiments', f"{word}:{sentiment}".encode('utf-8'))
        sent_words.add(word)
    
    print(f"Sent {len(batch)} words to 'sentiments' topic.")
    return True


In [2]:
# Load the file into an RDD and create key-value pairs (key, text)
file_path2 = "sentiment_text.csv"
text_rdd = sc.textFile(file_path2).map(lambda line: tuple(line.split(',')))

# Initialize Kafka Producer
producer = Producer({'bootstrap.servers': 'localhost:9092'}) 

# Set to keep track of already sent words
sent_keys = set()

def send_text():
    global text_rdd, sent_keys
    
    # Filter out already sent words
    unsent_rdd = text_rdd.filter(lambda kv: kv[0] not in sent_keys)
    
    # Collect data and pick 10 random keys and values
    if unsent_rdd.count() == 0:
        print("All sentences have been sent. Exiting.")
        return False  # Exit condition if all sentences have been sent
   
    # Randomly select 10 sentences
    batch = unsent_rdd.takeSample(withReplacement=False, num=10, seed=1)
        
    # Send batch to Kafka and mark it as sent
    for key, text in batch:
        sent_keys.add(key)
    producer.produce('text', f"{batch}".encode('utf-8'))
    print(f"Sent {len(batch)} sentences to 'text' topic.")
    return True

In [ ]:
import threading
import time
# Loop to send data every 2 seconds
while True:
    # Create threads for each function
    thread1 = threading.Thread(target=send_text)
    thread2 = threading.Thread(target=send_sentiments)

    # Start the first function and wait 2 seconds
    thread1.start()
    time.sleep(2)

    # While waiting, start the second function
    thread2.start()

    # Join threads to ensure they complete before restarting the loop
    thread1.join()
    thread2.join()

sc.stop()

Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words to 'sentiments' topic.
Sent 10 sentences to 'text' topic.
Sent 100 words t